# 출력층 설계 (Output Layer)

In [1]:
!pip3 install torch torchvision torchaudio

  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.22.1-cp312-cp312-win_amd64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.7.1-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.7.1-cp312-cp312-win_amd64.whl (216.1 MB)
Using cached torchvision-0.22.1-cp312-cp312-win_amd64.whl (1.7 MB)
Using cached torchaudio-2.7.1-cp312-cp312-win_amd64.whl (2.5 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Using cached fsspec-2025.7.0-py3-none-any.whl (199 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB

### Softmax

In [2]:
import numpy as np

def softmax(z):
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z)

# How? 비율은 똑같이 유지가 되니까 이런 계산식 적용이 가능 
def stable_softmax(z):
    exp_z = np.exp(z - np.max(z)) # Overflow prevention
    return exp_z / np.sum(exp_z)


x = np.array([1000, 1001, 1002])
# print(softmax(x)) # This causes overflow by e^z exceeding the max limit -> calculation error
print(stable_softmax(x))

[0.09003057 0.24472847 0.66524096]


- pythorch 라이브러리 함수 사용

In [4]:
import torch 
import torch.nn.functional as F

x = torch.tensor([1000, 1001, 1002], dtype = torch.float32) # pytorch식 텐서 만들기

softmax_output = F.softmax(x) # torch에서 제공하는 overflow 없는 stable한 softmax (torch의 tensor 타입만 가능, torch의 float만 가능)
print(softmax_output)

# pytorch에서 sigmoid는 functional 말고 torch에 있음
sigmoid_output = torch.sigmoid(x)
print(sigmoid_output)

tensor([0.0900, 0.2447, 0.6652])
tensor([1., 1., 1.])


/var/folders/mz/fktgp6vj62l5dw84vjcg4zj40000gn/T/ipykernel_29479/3246280021.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax_output = F.softmax(x) # torch에서 제공하는 overflow 없는 stable한 softmax (torch의 tensor 타입만 가능, torch의 float만 가능)


### 손실 함수와 연계

In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim

# 간단한 다중 클래스 분류 모델 정의
class SimpleMultiClassModel(nn.Module):
    def __init__(self):
        super(SimpleMultiClassModel, self).__init__()
        self.fc = nn.Linear(5,3) # fc = fully connected, Linear는 선형 계산 층 말하는 것 -> (5,3) 5개의 입력 받아서 3개의 출력

    # 순전파, scikit-learn에서 .predict 같은 기능
    def forward(self, x):
        return self.fc(x)
    

model = SimpleMultiClassModel()
criterion = nn.CrossEntropyLoss() # 손실함수 정의
optimizer = optim.Adam(model.parameters(), lr=0.01) # 가중치 최적화 함수 (adam) 정의

inputs = torch.randn(4,5) # (4,5) shape으로 표준 정규분포 형태 따르는 난수 생성
labels = torch.tensor([0, 2, 1, 0])

# 가중치 업데이트 시키면서 손실값이 줄어드는 것 확인
for _ in range(10):
    preds = model(inputs) # 순전파
    loss = criterion(preds, labels) # 손실계산
    print(loss.item())

    optimizer.zero_grad() # 기울기 초기화 (이전 단계에서 계산된 기울기를 0으로 초기화)
    loss.backward() # 역전파 (손실에 대한 역전파 수행 - 파라미터에 대한 기울기 계산)
    optimizer.step() # 가중치 업데이트 (계산된 기울기를 사용하여 옵티마이저가 모델 파라미터 갱신)

1.2759089469909668
1.238755702972412
1.2023335695266724
1.1666080951690674
1.1315937042236328
1.0973193645477295
1.06381094455719
1.0310912132263184
0.9991796612739563
0.9680922031402588
